https://www.youtube.com/watch?v=JRCJ6RtE3xU
https://medium.com/@sdoshi579/to-read-emails-and-download-attachments-in-python-6d7d6b60269
https://stackoverflow.com/questions/17874360/python-how-to-parse-the-body-from-a-raw-email-given-that-raw-email-does-not/32840516

In [1]:
from vars import vars

In [11]:
from email.message import EmailMessage
import email
import os
import smtplib
import imaplib
from mail_sender import sendmail

In [1]:
from utils.df_handle import *
import pandas
from google.cloud import bigquery
from datetime import datetime 
import xlsxwriter
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "D:\ipynb\spatial-vision-343005-340470c8d77b.json"


In [99]:
sql=f"""
-- Create view view_report.f_data_kh_checkin_canchamsoc
-- as

with data_kh_chua_viengtham as (SELECT slsperid,
mds,tenquanlytt,tenquanlykhuvuc,tenquanlyvung,custid,custname,statedescr,territorydescr,count(distinct soluong_checkin_thucte) 
  from `view_report.f_data_checkin_pbh` 

where  extract ( month from (select * from `biteam.d_current_table`)) = extract ( month from visitdate) 
and extract ( year from (select * from `biteam.d_current_table`)) = extract ( year from visitdate)
group by 1,2,3,4,5,6,7,8,9 having count(distinct soluong_checkin_thucte) =0),

data_gpp as (
 with data_quydinh_viengtham as (
SELECT distinct slsperid,custid,date_trunc(visitdate,month) as mapping_date FROM `spatial-vision-343005.biteam.sync_dms_salesroutedet`
where 
extract ( month from (select * from `biteam.d_current_table`)) = extract ( month from visitdate) 
and extract ( year from (select * from `biteam.d_current_table`)) = extract ( year from visitdate)
 )

select a.*,b.slsperid,d.tencvbh,
date_trunc ( thoihanhieulucgdpgpp,month) as mapping_date,
Case when  date(thoihanhieulucgdpgpp) < (select * from `biteam.d_current_table`) 
    then 1 else 0 end as is_hetthoihanhieuluc,
Case when 
     date_add((select * from `biteam.d_current_table`),interval 30 day) >= date(thoihanhieulucgdpgpp) and date(thoihanhieulucgdpgpp) > (select * from `biteam.d_current_table`) 
     then 1 else 0 end as is_saphetthoihanhieuluc,
Case when  date(thoihanhieulucgdpgpp) < (select * from `biteam.d_current_table`) 
    then makhachhangdms else null end as custid_hetthoihanhieuluc,
Case when 
     date_add((select * from `biteam.d_current_table`),interval 30 day) >= date(thoihanhieulucgdpgpp) and date(thoihanhieulucgdpgpp) > (select * from `biteam.d_current_table`) 
     then makhachhangdms else null end as custid_saphetthoihanhieuluc,
     c.custname,
     d.tenquanlytt,
     d.tenquanlykhuvuc,
     d.tenquanlyvung,
     c.statedescr,
     c.territorydescr
 from biteam.d_gpp a
LEFT JOIN data_quydinh_viengtham b on a.makhachhangdms = b.custid 
LEFT JOIN `biteam.d_master_khachhang` c on c.custid = a.makhachhangdms
LEFT JOIN `biteam.d_users` d on d.manv =b.slsperid

),

data_kh_quydinh_viengtham as (
SELECT slsperid,count(distinct custid)as so_kh_quydinh_viengtham FROM `spatial-vision-343005.biteam.sync_dms_salesroutedet`
where 
extract ( month from (select * from `biteam.d_current_table`)) = extract ( month from visitdate) 
and extract ( year from (select * from `biteam.d_current_table`)) = extract ( year from visitdate)
group by 1
 ),

group_data_kh_chua_viengtham as 
 (
   select slsperid,count(distinct custid) as so_kh_chua_viengtham 
   from data_kh_chua_viengtham
   group by 1
 ),
 group_data_gpp as 
 (
   select slsperid,count(distinct custid_saphetthoihanhieuluc) as custid_saphetthoihanhieuluc,
   count(distinct custid_hetthoihanhieuluc) as custid_hetthoihanhieuluc
   from data_gpp 
   group by 1
 )

 select  
  c.slsperid,
  d.tencvbh,
  c.so_kh_quydinh_viengtham,
  
  Case when a.so_kh_chua_viengtham is null then 0 else a.so_kh_chua_viengtham end as so_kh_chua_viengtham,
  Case when b.custid_saphetthoihanhieuluc is null then 0 else b.custid_saphetthoihanhieuluc end as custid_saphetthoihanhieuluc,
  Case when b.custid_hetthoihanhieuluc is null then 0 else b.custid_hetthoihanhieuluc end as custid_hetthoihanhieuluc,
  Case when c.slsperid ='MR1282' then 'nhanvt92@gmail.com'
        when c.slsperid ='MR1995' then 'nhanvt@merapgroup.com'
        when c.slsperid ='MR1605' then 'duyvq@merapgroup.com'
  else null end as send_email
 from data_kh_quydinh_viengtham c

 LEFT JOIN group_data_kh_chua_viengtham  a on c.slsperid = a.slsperid
 LEFT JOIN group_data_gpp b on c.slsperid =b.slsperid
 LEFT JOIN `biteam.d_users` d on d.manv =c.slsperid
 
  where c.slsperid in ('MR1282','MR1995','MR1605')

-- data_kh_phatsinh_dh as 
-- (
--     select slsperid,
-- -- date(visitdate) as visitdate,
-- mds as tencvbh,tenquanlytt,tenquanlykhuvuc,tenquanlyvung,custid,custname,statedescr,territorydescr,
--  count(distinct sl_kh_phatsinhdh) as sl_kh_phatsinh_dh
-- from `view_report.f_data_checkin_pbh`

-- where tuyen_kh is not null 
-- and  extract ( month from (select * from `biteam.d_current_table`)) = extract ( month from visitdate) 
-- and extract ( year from (select * from `biteam.d_current_table`)) = extract ( year from visitdate)
-- group by 1,2,3,4,5,6,7,8,9
-- having  sl_kh_phatsinh_dh > 0 )





"""

In [198]:
bigqueryClient = bigquery.Client()
df = bigqueryClient.query(sql).to_dataframe()
# df.index=blankIndex

In [199]:
date_now = datetime.now().date()

In [200]:
path=f'D:/ipynb/email_merap/_{date_now}_data_send_email.csv'

In [201]:
df.to_csv(path,index = False,header=False)

In [122]:
# with open(os.getcwd()+"\\_2022-09-30_data_send_email.csv",encoding='utf-8') as f:
#     line=f.readlines()[1:]
#     for line in line:
#         manv, tencvbh, so_kh_quydinh_viengtham, so_kh_chua_viengtham,custid_saphetthoihanhieuluc,custid_hetthoihanhieuluc,send_email = line.split(',')
#         print(manv, tencvbh, so_kh_quydinh_viengtham, so_kh_chua_viengtham,custid_saphetthoihanhieuluc,custid_hetthoihanhieuluc,send_email)

MR1282 Nguyễn Thị Khánh Vy 253 95 9 55 nhanvt92@gmail.com

MR1995 Nguyễn Thanh Thúy Liễu 185 100 3 57 nhanvt@merapgroup.com

MR1605 Nguyễn Chí Em 114 63 2 20 duyvq@merapgroup.com



In [210]:
with open(os.getcwd()+"\\_2022-09-30_data_send_email.csv",encoding='utf-8') as f:
    # line=f.readlines()[1:]
    # f.close()
    # index =0
    for i in range(10):
        manv, tencvbh, so_kh_quydinh_viengtham, so_kh_chua_viengtham,custid_saphetthoihanhieuluc,custid_hetthoihanhieuluc,send_email=f.readline().split(',')
        print(manv, tencvbh, so_kh_quydinh_viengtham, so_kh_chua_viengtham,custid_saphetthoihanhieuluc,custid_hetthoihanhieuluc,send_email)

MR1995 Nguyễn Thanh Thúy Liễu 185 100 3 57 nhanvt@merapgroup.com

MR1282 Nguyễn Thị Khánh Vy 253 95 9 55 nhanvt92@gmail.com

MR1605 Nguyễn Chí Em 114 63 2 20 duyvq@merapgroup.com



ValueError: not enough values to unpack (expected 7, got 1)

In [ ]:
# for i in range(1,200):
#     #Email connection
#     EMAIL_ADDRESS = 'bi@merapgroup.com'
#     EMAIL_PASSOWRD = 'Bimerap@999'
#     msg = EmailMessage()
#     # FROM EMAIL
#     msg['From'] = EMAIL_ADDRESS
#     # TO EMAIL
#     msg['To'] = ['duyvq@merapgroup.com', 'bi@merapgroup.com']
#     # BODY
#     msg.set_content('DATA VALIDATION ALERT FROM BI TEAM')
#     msg['Subject'] = f'BI BOT - Data Driven Reports 2022-09-22-{i}'
#     with smtplib.SMTP_SSL('mail.merapgroup.com', 465) as smtp:
#         conn_smtp = smtp.login(EMAIL_ADDRESS, EMAIL_PASSOWRD)
#         print(conn_smtp)
#         smtp.send_message(msg)
#         print('Message has been sent')

In [3]:
# with open(os.getcwd()+"\\_2022-09-30_data_send_email.csv") as f:
#     for i in range(1,200):
#         try:
#             manv, tencvbh, so_kh_quydinh_viengtham, so_kh_chua_viengtham,custid_saphetthoihanhieuluc,custid_hetthoihanhieuluc,send_email = f.readline().split(',')
#             sendmail(manv, tencvbh, so_kh_quydinh_viengtham, so_kh_chua_viengtham,custid_saphetthoihanhieuluc,custid_hetthoihanhieuluc,send_email)
#         except:
#             break

In [104]:
import pandas as pd

In [202]:
from datetime import datetime
from email.header import Header

# from email import utils
# from email import errors
# from email._policybase import Policy, compat32
# from email import charset as _charset
# from email._encoded_words import decode_b
# Charset = _charset.Charset

In [211]:
def sendmail(manv, tencvbh, so_kh_quydinh_viengtham, so_kh_chua_viengtham,custid_saphetthoihanhieuluc,custid_hetthoihanhieuluc,send_email):
#Email connection
    EMAIL_ADDRESS = 'bi@merapgroup.com'
    EMAIL_PASSOWRD = 'Bimerap@999'
    msg = EmailMessage()
     #Subject
    msg['Subject'] = f'[Data Driven Report] Danh Sách Khách Hàng Cần Quan Tâm ngày {date_now}'
     # FROM EMAIL
    msg['From'] = EMAIL_ADDRESS
        # TO EMAIL
    msg['To'] = send_email
        # BODY
    msg.set_content(f'''
    Xin Chào: {tencvbh}, Mã số NV: {manv}

    1) Bạn Còn {so_kh_chua_viengtham} / {so_kh_quydinh_viengtham} Nhà Thuốc chưa viếng thăm trong tháng này:
    Link: https://datastudio.google.com/s/tZDbi53fhZ0

    2) Bạn có {custid_saphetthoihanhieuluc} Nhà Thuốc sắp hết thởi gian GPP trong tháng:
    Link: https://datastudio.google.com/s/tZDbi53fhZ0

    3) Bạn có {custid_hetthoihanhieuluc} Nhà Thuốc sắp hết thởi gian GPP trong tháng:
    Link: https://datastudio.google.com/s/tZDbi53fhZ0

    Đây là tin nhắn gửi tự động vui lòng không phản hồi !!!
    ''')
    with smtplib.SMTP_SSL('mail.merapgroup.com', 465) as smtp:
        conn_smtp = smtp.login(EMAIL_ADDRESS, EMAIL_PASSOWRD)
        print(conn_smtp)
        smtp.send_message(msg)
        print('Message has been sent')
        print(datetime.now())
    print(send_email)

In [212]:
import time
def send_email_auto():
    with open(os.getcwd()+"\\_2022-09-30_data_send_email.csv",encoding='utf-8') as f:
        # line=f.readlines()[1:]
        # f.close()
        for i in range(10):
            try:
                manv, tencvbh, so_kh_quydinh_viengtham, so_kh_chua_viengtham,custid_saphetthoihanhieuluc,custid_hetthoihanhieuluc,send_email = f.readline().split(',')
                sendmail(manv, tencvbh, so_kh_quydinh_viengtham, so_kh_chua_viengtham,custid_saphetthoihanhieuluc,custid_hetthoihanhieuluc,send_email)
                time.sleep(5)
            except:
                break


In [213]:
send_email_auto()

(235, b'2.7.0 Authentication successful')
Message has been sent
2022-09-30 14:42:57.003406
nhanvt@merapgroup.com

(235, b'2.7.0 Authentication successful')
Message has been sent
2022-09-30 14:43:02.215070
nhanvt92@gmail.com

(235, b'2.7.0 Authentication successful')
Message has been sent
2022-09-30 14:43:07.377331
duyvq@merapgroup.com



In [92]:
# def send_email_auto():
#     with open(os.getcwd()+"\\_2022-09-30_data_send_email.csv") as f:
#         lines = f.readlines()[1:]
#         for i in range(2,200):
#             try:
#                 manv, tencvbh, sokh, so_khchua,saphethan,hetthoihan,send_email = f.readline().split(',')
#                 sendmail(manv, tencvbh, sokh, so_khchua,saphethan,hetthoihan,send_email )
#             except:
#                 break

In [93]:
send_email_auto()

In [57]:
# for i in range(1,200):
#Email connection
from datetime import datetime
index =0
for i in to_email:
    EMAIL_ADDRESS = 'bi@merapgroup.com'
    EMAIL_PASSOWRD = 'Bimerap@999'
    msg = EmailMessage()
    # FROM EMAIL
    msg['From'] = EMAIL_ADDRESS
    # TO EMAIL
    msg['To'] = i
    print(index, i)
    index += 1
    # BODY
    msg.set_content(f''' 
    Xin Chào {tencvbh}, Mã số NV {manv}

    1) Bạn Còn {so_kh_chua_viengtham} / {so_kh_quydinh_viengtham} Nhà Thuốc chưa viếng thăm trong tháng này:
    Link: https://datastudio.google.com/s/tZDbi53fhZ0

    # 2) Bạn có {df['custid_saphetthoihanhieuluc']} Nhà Thuốc sắp hết thởi gian GPP trong tháng:
    # Link: https://datastudio.google.com/s/tZDbi53fhZ0

    # 3) Bạn có {df['custid_hetthoihanhieuluc']} Nhà Thuốc sắp hết thởi gian GPP trong tháng:
    # Link: https://datastudio.google.com/s/tZDbi53fhZ0

    Đây là tin nhắn gửi tự động vui lòng không phản hồi !!!
    '''
    # , subtype='text/plain'
    )

    msg['Subject'] = f'[Data Driven Report] Danh Sách Khách Hàng Cần Quan Tâm ngày 2022-09-30'
    with smtplib.SMTP_SSL('mail.merapgroup.com', 465) as smtp:
        conn_smtp = smtp.login(EMAIL_ADDRESS, EMAIL_PASSOWRD)
        print(conn_smtp)
        smtp.send_message(msg)
        print('Message has been sent')
        print(datetime.now())

0 nhanvt@merapgroup.com
(235, b'2.7.0 Authentication successful')
Message has been sent
2022-09-30 12:30:09.679202
1 nhanvt92@gmail.com
(235, b'2.7.0 Authentication successful')
Message has been sent
2022-09-30 12:30:09.814205
2 duyvq@merapgroup.com
(235, b'2.7.0 Authentication successful')
Message has been sent
2022-09-30 12:30:09.950230


In [155]:
# #Email connection
# EMAIL_ADDRESS = 'report@merapgroup.com'
# EMAIL_PASSOWRD = 'Report@111'
# msg = EmailMessage()
# # FROM EMAIL
# msg['From'] = EMAIL_ADDRESS
# # TO EMAIL
# msg['To'] = ['duyvq@merapgroup.com']
# # BODY
# msg.set_content('DATA VALIDATION ALERT FROM BI TEAM')
# msg['Subject'] = '[ALERT] IN TRANSIT df has duplicate values'
# with imaplib.IMAP4_SSL('smtp.merapgroup.com') as conn:
#     conn.login(EMAIL_ADDRESS, EMAIL_PASSOWRD)
#     conn.select('INBOX')
#     print(conn.select('INBOX'))
#     result, data = conn.uid('search', None, "ALL")
#     print(data[0])
#     inbox_item_list = data[0].split()
#     print(inbox_item_list)
#     newest = inbox_item_list[-1]
#     result2, email_data = conn.uid('fetch', newest, '(RFC822)')
#     raw_email = email_data[0][1].decode('utf-8')
#     email_message = email.message_from_string(raw_email,policy=email.policy.default)
#     print(dir(email_message))
#     print('====')
#     print(email_message['To'])
#     print('====')
#     print(email_message['From'])
#     print('====')
#     print(email_message['Subject'])
#     print('====')
#     print(email_message.__contains__)
#     print('====')
#     print(email_message.get_content_maintype())
#     print('====')
#     print(email_message.walk())
# #     for part in email_message.walk():
# #         print(part)

('OK', [b'12'])
b'1 2 3 4 5 6 7 8 9 10 11 12'
[b'1', b'2', b'3', b'4', b'5', b'6', b'7', b'8', b'9', b'10', b'11', b'12']
['__bytes__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_add_multipart', '_body_types', '_charset', '_default_type', '_find_body', '_get_params_preserve', '_headers', '_make_multipart', '_payload', '_unixfrom', 'add_alternative', 'add_attachment', 'add_header', 'add_related', 'as_bytes', 'as_string', 'attach', 'clear', 'clear_content', 'defects', 'del_param', 'epilogue', 'get', 'get_all', 'get_body', 'get_boundary', 'get_charset', 'get_charsets', 'get_content', 'get_con

In [148]:
print(email_message['Subject'])

TEST SUBJECT


In [156]:
body = email_message.get_body(('plain',))
if body:
    body = body.get_content()
print(body)

Hi this is a body of the email

 




In [150]:
email_message.is_multipart()

True

In [133]:
list = []
for payload in email_message.get_payload():
    print(payload.get_payload())

[<email.message.Message object at 0x000001C9A90E4550>, <email.message.Message object at 0x000001C9A90E4820>]
import os=0A=
from googleapiclient import discovery=0A=
from google.oauth2 import service_account=0A=
import pandas as pd=0A=
import warnings=0A=
import psycopg2=0A=
from urllib.parse import urlparse=0A=
from collector.libs.pandas_utils import insert_df_to_postgres=0A=
from email.message import EmailMessage=0A=
=0A=
warnings.filterwarnings('ignore')=0A=
=0A=
=0A=
def get_postgres_conn(url):=0A=
    try:=0A=
        result =3D urlparse(url)=0A=
        username =3D result.username=0A=
        password =3D result.password=0A=
        database =3D result.path[1:]=0A=
        hostname =3D result.hostname=0A=
        port =3D result.port=0A=
=0A=
        return psycopg2.connect(user=3Dusername,=0A=
                                password=3Dpassword,=0A=
                                host=3Dhostname,=0A=
                                port=3Dport,=0A=
                             

In [136]:
os.getcwd()

'C:\\Users\\DELL\\Downloads\\Documents\\ipython_file'

In [158]:
# downloading attachments
for part in email_message.walk():
    # this part comes from the snipped I don't understand yet... 
    if part.get_content_maintype() == 'multipart':
        continue
    if part.get('Content-Disposition') is None:
        continue
    fileName = part.get_filename()
    if bool(fileName):
        filePath = os.path.join(os.getcwd(), fileName)
        if not os.path.isfile(filePath) :
            fp = open(filePath, 'wb')
            fp.write(part.get_payload(decode=True))
            fp.close()
#             subject = str(email_message).split("Subject: ", 1)[1].split("\nTo:", 1)[0]
#             print('Downloaded "{file}" from email titled "{subject}" with UID {uid}.'.format(file=fileName, subject=subject, uid=newest))